<a href="https://colab.research.google.com/github/hongwon2/BigQuery-Practice/blob/main/RFM_%EB%B6%84%EC%84%9D_%EC%A7%80%ED%91%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RFM 분석 
**고객 분류를 위한 중요한 분석 방법**
## RFM 분석에서는 다음과 같은 3가지 지표를 기반으로 사용자를 그룹화 한다.
- Recency : 최근 구매일
  - 최근 무언가를 구매한 사용자를 우량 고객으로 취급
- Frequency : 구매 횟수
  - 사용자가 구매한 횟수를 세고, 많을수록 우량 고객으로 취급
- Monetary : 구매 금액 합계

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%load_ext google.colab.data_table

### 샘플 데이터

In [6]:
%%bigquery --project sql-recipe2
select *
from ch5.action_log
order by session

,session,user_id,action,category,products,amount,stamp
0,098vhaoi,U008,add_cart,SF,A002,NaN,2016-12-04 12:00:00
1,098vhaoi,U008,add_cart,SF,A007,NaN,2016-12-04 12:00:00
2,098vhaoi,U008,add_cart,SF,A007,NaN,2016-12-04 12:00:00
3,111ii111,U007,purchase,drama,D002,1000.0,2016-11-12 13:00:00
4,111ii111,U007,purchase,action,"A005,A006",1000.0,2016-11-12 15:00:00
...,...,...,...,...,...,...,...
58,asdv8888,U007,add_cart,drama,D001,NaN,2016-12-03 19:00:00
59,asdv8888,U007,add_cart,drama,D002,NaN,2016-12-03 20:30:00
60,asdv8888,U007,purchase,drama,D001,1000.0,2016-12-03 20:00:00
61,ndkn1100,U009,purchase,drama,D002,1000.0,2016-12-05 13:00:00


### 사용자 별로 RFM 을 집계하는 쿼리

In [7]:
%%bigquery --project sql-recipe2
with
purchase_log as (
  select
    user_id,
    amount,
    substr(stamp,1,10) as dt
  from
    `sql-recipe2.ch5.action_log`
  where
  action = 'purchase'
)
,user_rfm as (
  select
    user_id,
    max(dt) as recent_date,
    date_diff(CURRENT_DATE, date(timestamp(MAX(dt))),day) as recency,
    count(dt) as frequency,
    sum(amount) as monetary
  from
    purchase_log
  group by
    user_id
)
select *
from user_rfm
order by user_id

,user_id,recent_date,recency,frequency,monetary
0,U001,2016-11-03,2058,2,4000
1,U002,2016-11-11,2050,1,1000
2,U004,2016-11-13,2048,2,2000
3,U005,2016-10-18,2074,2,4000
4,U006,2016-11-09,2052,1,1000
5,U007,2016-12-03,2028,8,10000
6,U009,2016-12-05,2026,2,2000


### 사용자들의 RFM 랭크를 계산하는 쿼리

In [10]:
%%bigquery --project sql-recipe2
with
purchase_log as (
  select
    user_id,
    amount,
    substr(stamp,1,10) as dt
  from
    `sql-recipe2.ch5.action_log`
  where
  action = 'purchase'
)
,user_rfm as (
  select
    user_id,
    max(dt) as recent_date,
    date_diff(CURRENT_DATE, date(timestamp(MAX(dt))),day) as recency,
    count(dt) as frequency,
    sum(amount) as monetary
  from
    purchase_log
  group by
    user_id
)
,user_rfm_rank as (
  select
    user_id,
    recent_date,
    recency,
    frequency,
    monetary,
    case
      when recency < 14 then 5
      when recency < 28 then 4
      when recency < 60 then 3
      when recency < 90 then 2
      else 1
    end as r,
    case
      when frequency >= 20 then 5
      when frequency >= 10 then 4
      when frequency >= 5 then 3
      when frequency >= 2 then 2
      when frequency = 1 then 1
    end as f,
    case
      when monetary >= 300000 then 5
      when monetary >= 100000 then 4
      when monetary >= 30000 then 3
      when monetary >= 50000 then 2
      else 1
    end as m
  from
    user_rfm
)
select *
from
  user_rfm_rank

,user_id,recent_date,recency,frequency,monetary,r,f,m
0,U005,2016-10-18,2074,2,4000,1,2,1
1,U001,2016-11-03,2058,2,4000,1,2,1
2,U007,2016-12-03,2028,8,10000,1,3,1
3,U006,2016-11-09,2052,1,1000,1,1,1
4,U002,2016-11-11,2050,1,1000,1,1,1
5,U004,2016-11-13,2048,2,2000,1,2,1
6,U009,2016-12-05,2026,2,2000,1,2,1


### 각 그룹의 사람 수를 확인하는 쿼리

In [11]:
%%bigquery --project sql-recipe2
#사용자 별로 rfm을 집계하는 쿼리
with
purchase_log as (
  select
    user_id,
    amount,
    substr(stamp,1,10) as dt
  from
    `sql-recipe2.ch5.action_log`
  where
  action = 'purchase'
)
,user_rfm as (
  select
    user_id,
    max(dt) as recent_date,
    date_diff(CURRENT_DATE, date(timestamp(MAX(dt))),day) as recency,
    count(dt) as frequency,
    sum(amount) as monetary
  from
    purchase_log
  group by
    user_id
)
#사용자들의 RFM 랭크를 계산하는 쿼리
,user_rfm_rank as (
  select
    user_id,
    recent_date,
    recency,
    frequency,
    monetary,
    case
      when recency < 14 then 5
      when recency < 28 then 4
      when recency < 60 then 3
      when recency < 90 then 2
      else 1
    end as r,
    case
      when frequency >= 20 then 5
      when frequency >= 10 then 4
      when frequency >= 5 then 3
      when frequency >= 2 then 2
      when frequency = 1 then 1
    end as f,
    case
      when monetary >= 300000 then 5
      when monetary >= 100000 then 4
      when monetary >= 30000 then 3
      when monetary >= 50000 then 2
      else 1
    end as m
  from
    user_rfm
)
# 각 그룹의 사람 수를 확인하는 쿼리
,mst_rfm_index as (
            select 1 as rfm_index
  union all select 2 as rfm_index
  union all select 3 as rfm_index
  union all select 4 as rfm_index
  union all select 5 as rfm_index
)
,rfm_flag as (
  select
    m.rfm_index,
    case when m.rfm_index = r.r then 1 else 0 end as r_flag,
    case when m.rfm_index = r.f then 1 else 0 end as f_flag,
    case when m.rfm_index = r.m then 1 else 0 end as m_flag,
  from
    mst_rfm_index as m
    cross join
    user_rfm_rank as r
)
select
  rfm_index,
  sum(r_flag) as r,
  sum(f_flag) as f,
  sum(m_flag) as m
from
  rfm_flag
group by
  rfm_index
order by
  rfm_index desc

,rfm_index,r,f,m
0,5,0,0,0
1,4,0,0,0
2,3,0,1,0
3,2,0,4,0
4,1,7,2,7


### 통합 랭크를 계산하는 쿼리 - RFM 분석을 1차원으로 나타내기

In [12]:
%%bigquery --project sql-recipe2
with
purchase_log as (
  select
    user_id,
    amount,
    substr(stamp,1,10) as dt
  from
    `sql-recipe2.ch5.action_log`
  where
  action = 'purchase'
)
,user_rfm as (
  select
    user_id,
    max(dt) as recent_date,
    date_diff(CURRENT_DATE, date(timestamp(MAX(dt))),day) as recency,
    count(dt) as frequency,
    sum(amount) as monetary
  from
    purchase_log
  group by
    user_id
)
,user_rfm_rank as (
  select
    user_id,
    recent_date,
    recency,
    frequency,
    monetary,
    case
      when recency < 14 then 5
      when recency < 28 then 4
      when recency < 60 then 3
      when recency < 90 then 2
      else 1
    end as r,
    case
      when frequency >= 20 then 5
      when frequency >= 10 then 4
      when frequency >= 5 then 3
      when frequency >= 2 then 2
      when frequency = 1 then 1
    end as f,
    case
      when monetary >= 300000 then 5
      when monetary >= 100000 then 4
      when monetary >= 30000 then 3
      when monetary >= 50000 then 2
      else 1
    end as m
  from
    user_rfm
)
select
  r + f + m as total_rank,
  r,
  f,
  m,
  count(user_id) as count
from 
  user_rfm_rank
group by
  r,f,m
order by
  total_rank desc, r desc, f desc, m desc;

,total_rank,r,f,m,count
0,5,1,3,1,1
1,4,1,2,1,4
2,3,1,1,1,2


### 종합 랭크별로 사용자 수를 집계하는 쿼리

In [13]:
%%bigquery --project sql-recipe2
with
purchase_log as (
  select
    user_id,
    amount,
    substr(stamp,1,10) as dt
  from
    `sql-recipe2.ch5.action_log`
  where
  action = 'purchase'
)
,user_rfm as (
  select
    user_id,
    max(dt) as recent_date,
    date_diff(CURRENT_DATE, date(timestamp(MAX(dt))),day) as recency,
    count(dt) as frequency,
    sum(amount) as monetary
  from
    purchase_log
  group by
    user_id
)
,user_rfm_rank as (
  select
    user_id,
    recent_date,
    recency,
    frequency,
    monetary,
    case
      when recency < 14 then 5
      when recency < 28 then 4
      when recency < 60 then 3
      when recency < 90 then 2
      else 1
    end as r,
    case
      when frequency >= 20 then 5
      when frequency >= 10 then 4
      when frequency >= 5 then 3
      when frequency >= 2 then 2
      when frequency = 1 then 1
    end as f,
    case
      when monetary >= 300000 then 5
      when monetary >= 100000 then 4
      when monetary >= 30000 then 3
      when monetary >= 50000 then 2
      else 1
    end as m
  from
    user_rfm
)
select
  r + f + m as total_rank,
  count(user_id) as count
from 
  user_rfm_rank
group by
  total_rank
order by
  total_rank desc

,total_rank,count
0,5,1
1,4,4
2,3,2


###R과F를 사용해 2차원 사용자의 수를 집계하기


In [14]:
%%bigquery --project sql-recipe2
with
purchase_log as (
  select
    user_id,
    amount,
    substr(stamp,1,10) as dt
  from
    `sql-recipe2.ch5.action_log`
  where
  action = 'purchase'
)
,user_rfm as (
  select
    user_id,
    max(dt) as recent_date,
    date_diff(CURRENT_DATE, date(timestamp(MAX(dt))),day) as recency,
    count(dt) as frequency,
    sum(amount) as monetary
  from
    purchase_log
  group by
    user_id
)
,user_rfm_rank as (
  select
    user_id,
    recent_date,
    recency,
    frequency,
    monetary,
    case
      when recency < 14 then 5
      when recency < 28 then 4
      when recency < 60 then 3
      when recency < 90 then 2
      else 1
    end as r,
    case
      when frequency >= 20 then 5
      when frequency >= 10 then 4
      when frequency >= 5 then 3
      when frequency >= 2 then 2
      when frequency = 1 then 1
    end as f,
    case
      when monetary >= 300000 then 5
      when monetary >= 100000 then 4
      when monetary >= 30000 then 3
      when monetary >= 50000 then 2
      else 1
    end as m
  from
    user_rfm
)
select
  concat('r_',cast(r as string)) as r_rank,
  count(case when f = 5 then 1 end) as f_5,
  count(case when f = 5 then 1 end) as f_4,
  count(case when f = 5 then 1 end) as f_3,
  count(case when f = 5 then 1 end) as f_2,
  count(case when f = 5 then 1 end) as f_1,
from 
user_rfm_rank
group by
r
order by
r_rank desc;

,r_rank,f_5,f_4,f_3,f_2,f_1
0,r_1,0,0,0,0,0
